In [6]:
import json
import random

In [ ]:
# PREPARA ARQUIVO DE TREINAMENTO COM OS DADOS DOS PRODUTOS

# Carregar o arquivo JSON com os dados dos produtos processados
with open("../../data/processed/products_data.json", "r") as file:
    products = json.load(file)

# Inicializar o conjunto de treinamento
TRAINING_DATA = []

for product in products:
    # Extrair as informações necessárias com verificações
    name = product.get("productName", "Produto desconhecido")
    brand = product.get("brand", "Marca desconhecida")
    categories = product.get("categories", ["Categoria desconhecida"])

    # Verificar a presença do preço com segurança
    price = None
    if "sellers" in product and product["sellers"]:
        commertial_offer = product["sellers"][0].get("commertialOffer")
        if commertial_offer:
            installment = commertial_offer.get("Installment")
            if isinstance(installment, dict):  # Verifica se Installment é um dicionário
                price = installment.get("Value")

    # Verificar se productClusters existe e extrair seus valores, caso contrário usar um placeholder
    clusters = product.get("productClusters", {})
    cluster_text = ", ".join(clusters.values()) if clusters else "Nenhum cluster disponível"
    
    # Construir a frase para treinamento, incluindo todas as categorias e ajustando a presença de preço
    categories_text = ", ".join(categories)
    if price is not None:
        phrase = f"{name} da marca {brand}, nas categorias {categories_text}, classificado como {cluster_text}, em oferta por R$ {price:.2f}."
    else:
        phrase = f"{name} da marca {brand}, nas categorias {categories_text}, classificado como {cluster_text}."

    # Identificar as posições das entidades na frase
    entities = [
        (phrase.find(name), phrase.find(name) + len(name), "PRODUCT_NAME"),
        (phrase.find(brand), phrase.find(brand) + len(brand), "BRAND")
    ]
    
    # Adicionar cada termo de categoria como uma entidade "CATEGORY"
    for category in categories:
        category_start = phrase.find(category)
        if category_start != -1:
            entities.append((category_start, category_start + len(category), "CATEGORY"))

    # Adicionar cada cluster como uma entidade "CLUSTER"
    cluster_start = phrase.find(cluster_text)
    if cluster_start != -1:
        entities.append((cluster_start, cluster_start + len(cluster_text), "CLUSTER"))
    
    # Adicionar a entidade PRICE se o preço estiver presente
    if price is not None:
        price_text = f"R$ {price:.2f}"
        price_start = phrase.find(price_text)
        entities.append((price_start, price_start + len(price_text), "PRICE"))

    # Adicionar a frase processada ao conjunto de dados de treinamento
    TRAINING_DATA.append((phrase, {"entities": entities}))

# Salvar o conjunto de treinamento em um arquivo JSON
with open("../../model/training_data/training_data.json", "w") as out_file:
    json.dump(TRAINING_DATA, out_file, indent=4, ensure_ascii=False)

print("Arquivo de treinamento 'training_data.json' gerado com sucesso.")


Arquivo de treinamento 'training_data.json' gerado com sucesso.


In [ ]:
# LIDA COM O ARRAY "CATEGORIES" E FORMATA DE FORMA IDEAL PARA USO NO TREINAMENTO

# Caminho para o arquivo de produtos
products_data_path = "../../data/processed/products_data.json"
processed_data_path = "../../data/processed/products_data.json"

# Carregar os dados dos produtos
with open(products_data_path, "r") as file:
    products = json.load(file)

# Função para processar e extrair categorias únicas
def process_categories(categories):
    unique_terms = set()
    for category_path in categories:
        # Dividir a categoria nos termos individuais, ignorando barras vazias
        terms = category_path.strip("/").split("/")
        unique_terms.update(terms)
    return list(unique_terms)

# Processar cada produto e atualizar as categorias
for product in products:
    categories = product.get("categories", [])
    product["categories"] = process_categories(categories)

# Salvar o novo conjunto de dados com categorias processadas
with open(processed_data_path, "w") as out_file:
    json.dump(products, out_file, indent=4, ensure_ascii=False)

print("Dados de categorias processados e salvos em 'processed_products_data.json'")


In [8]:
# DIVIDE OS DADOS DOS PRODUTOS PARA TREINAMENTO E TESTES

# Caminho para o arquivo de dados de treinamento
training_data_path = "../../model/training_data/training_data.json"

# Carregar os dados do arquivo de treinamento
with open(training_data_path, "r") as file:
    all_data = json.load(file)

# Embaralhar os dados para garantir a aleatoriedade
random.shuffle(all_data)

# Dividir 80% para novo conjunto de treinamento e 20% para validação
split_index = int(0.8 * len(all_data))
new_training_data = all_data[:split_index]
validation_data = all_data[split_index:]

# Salvar o novo conjunto de treinamento e o conjunto de validação
with open(training_data_path, "w") as train_file:
    json.dump(new_training_data, train_file, indent=4, ensure_ascii=False)

validation_data_path = "../../data/processed/validation_data.json"
with open(validation_data_path, "w") as val_file:
    json.dump(validation_data, val_file, indent=4, ensure_ascii=False)

print("Arquivos 'training_data.json' e 'validation_data.json' foram gerados com sucesso.")

Arquivos 'training_data.json' e 'validation_data.json' foram gerados com sucesso.
